**Carga las librerías de nummpy, pandas y matplotlib con el nombre de siempre**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Lee los datos adjuntos a este notebook, creando un dataframe de pandas con el nombre que quieras. Ajusta el argumento na_values a '?' ****

In [ ]:
dat_weather = pd.read_csv(filepath_or_buffer="data_2.csv")

**Redefine el dataframe anterior quitando las columnas ["Date","Location", "RainToday", "RISK_MM"]**

In [ ]:
dat_weather.drop(columns=["Date","Location", "RainToday", "RISK_MM"], 
                 inplace = True)

**La data contiene huecos. Quítalos**

In [ ]:
dat_weather.dropna(inplace=True)

In [ ]:
dat_weather.shape

**Las variables 'WindGustDir', 'WindDir3pm', 'WindDir9am' son categóricas. Crea dummy variables, esto es, que sean ortogonales usando un one-hot-encoding mediante la funcionalidad `get_dummies` de pandas**

In [ ]:
categorical_columns = ['WindGustDir', 'WindDir3pm', 'WindDir9am']

In [ ]:
dat_weather_proc = pd.get_dummies(dat_weather, 
                                  columns=categorical_columns)

**A partir de esta dataset, define una matrix de features X y un vector de labels y**

In [ ]:
X = dat_weather_proc.drop(columns=['RainTomorrow']).values
y = dat_weather_proc.loc[:, 'RainTomorrow'].values

**Codifica el vector de labels a 0 y 1 usando `LabelEncoder`**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

**Divide el 80% de los datos para training y el 20% restante para test. Para ello usa la función `train_test_split`, fijando el random_state=0**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=0,
                                                    test_size=0.2)

**Importa un clasificador tipo arbol de decision, y crea un objeto de esta clase. Cuando definas el objeto, elige el argumento `random_state` igual a 0, `class_weight` igual a 'balanced'**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0, 
                             class_weight='balanced')

**Ajusta el modelo sobre el train y calcula la accuracy sobre el mismo training y test**

In [ ]:
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

**Después de ajustar el clasificador, éste da un atributo llamado "feature_importances_" que da un vector con la importancia de cada feature. Plotea la importancia de cada feature usando barras. Puedes usar la function bar dentro de matplotlib. Si tienes alguna duda de cómo usar la función, puedes mirar la documentación**

In [ ]:
plt.bar(np.arange(len(clf.feature_importances_)),
        clf.feature_importances_)

**Vamos a calcular lo anterior para diferentes particiones de los datos. Para ello, primero importa la clase `StratifiedKFold` contenida en el módulo de model_selection. Después, define una variable que sea un objeto de esta clase y que implemente un 5-Fold cross-validation. Fija random_state=0**

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, random_state=0)

**Calcula la accuracy promedio a través de estos 5 folds**

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(X=X, y=y, estimator = clf, cv=cv).mean()

**Hay muchas variables, ¿verdad? Puede que no se necesiten todas. Vamos a realizar un feature selection basado en las importancias dadas por el árbol de decision anterior. El clasificador final será en este caso `Logistic Regression`. Por ello, primero importa la clase que implementa este algoritmo y define un objeto de dicho clasificador al que llamaremos "log_clf". Elige random_state=0 y class_weight='balanced' cuando definas este objeto**

In [ ]:
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression(class_weight='balanced', random_state=0)

**Importa la clase `SelectFromModel`, que vamos a usar para seleccionar las features más importantes basado en la importancia de las features en el árbol de decision. Define un objeto de la clase recién importada y llámala "feat". No olvides que dicho objeto requiere que le pases el árbol de decision que va a usar para estimar la importancia de la features a seleccionar. Si tienes alguna duda, mira en la documentación**

In [ ]:
from sklearn.feature_selection import SelectFromModel
feat = SelectFromModel(estimator=DecisionTreeClassifier(class_weight='balanced', 
                                                        random_state=0))

**Además, algunas features tienen escala diferente. Por ello, importa la clase `MinMaxScaler` y define un objeto a partir de esta clase, llamándola "scaler"**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

**Vamos a encadenar los tres objetos, que implementan diferentes operaciones, en un sólo objeto. Para ello, importa la clase `Pipeline` y crea un objeto a partir de ella llamada "pip". Los pasos van en este orden: primero el reescalado de los datos, después feature selection y por último el clasificador**

In [ ]:
from sklearn.pipeline import Pipeline
pip = Pipeline([('scaler', scaler), 
                ('feat', feat), 
                ('clf', log_clf)])

**Calcula la accuracy promediada de este objeto tipo pipeline usado el cross-validation definido más arriba**

In [ ]:
cross_val_score(estimator=pip, X=X, y=y, cv=cv).mean()

**A veces algunos clasificadores ya implementan esto por construcción y dan un rendimiento mejor. Prueba esto con un clasificador de tipo Random Forest, implementada con el nombre `RandomForestClassifier`. Usa 50 árboles de decisión y acuérdate de elegir random_state=0 y class_weight='balanced'**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(class_weight='balanced', 
                             random_state=0,
                             n_estimators=50)

cross_val_score(clf, X, y, cv = 5).mean()

**(Extra) Repite esto último con la dataset inicial, pero rellenando los NaNs en cada columna con la mediana en sus valores observados**

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(strategy = 'median')

pip = Pipeline([('imputer', imp), ('clf', clf)])

In [ ]:
dat_weather_all = pd.read_csv(filepath_or_buffer="weatherAUS.csv")
dat_weather_all.drop(columns=["Date","Location", "RainToday", "RISK_MM"], 
                 inplace = True)
dat_weather_all_proc = pd.get_dummies(dat_weather_all, 
                                      columns=categorical_columns)

X_all = dat_weather_all_proc.drop(columns=['RainTomorrow']).values
y_all = dat_weather_all_proc.loc[:, 'RainTomorrow'].values

In [ ]:
cross_val_score(pip, X_all, y_all, cv = 5).mean()